# Guide
실행 불필요  
전처리 완료된 데이터 링크 아래 참고
---

# Data
[[Google Drive] /Data/Initial/Step1.zip](https://drive.google.com/file/d/1LpnhjeBOoLH-QwbVOovCk-OpdpmIQAaZ/view?usp=sharing)

## Dataset Source
[[Kaggle] Fruit and Vegetable Disease (Healthy vs Rotten)](https://www.kaggle.com/datasets/muhammad0subhan/fruit-and-vegetable-disease-healthy-vs-rotten)  
apple 데이터만 사용

## Preprocessing
기존 데이터셋에는 이미 증강 이미지 존재(일부 클래스에만) → 다른 Step의 모델에도 사용하기 위해 증강 이미지 삭제 + 중복 이미지 삭제  

---

# Data Augmentation
`ImageDataGenerator` 사용

## Augmentation Ratio
1개의 이미지 당 2개의 증강 이미지 생성

## Data
|              |&nbsp;&nbsp;&nbsp;Original&nbsp;&nbsp;&nbsp;| Augmentation |&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Final&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;|
|:------------:|:------------:|:------------:|:------------:|
| Apple_Fresh  | 368 | 728 | 1096 |
| Apple_Rotten | 462 | 917 | 1379 |



In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import os
import numpy as np

2024-11-20 13:54:34.898288: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
datagen = ImageDataGenerator(
    rotation_range=90,            # 회전 각도 범위 [-90, +90]
    shear_range=0.3,              # 전단 변형(층밀림) 각도 [0, 0.3](radian):최대 약 17도
    zoom_range=[0.7, 1.3],        # 확대/축소 범위. 원본의 70% ~ 130%
    brightness_range=[0.2, 1.3],  # 밝기 조정 범위. 원본의 20% ~ 130%
    horizontal_flip=True,         # 좌우 반전
    vertical_flip=True,           # 상하 반전
)

In [3]:
def data_augmentation(input_dir, output_dir):
    
    for filename in os.listdir(input_dir):
        
        if filename.endswith(".jpg") or filename.endswith(".png"):
            
            img_path = os.path.join(input_dir, filename)
            img = load_img(img_path)  # 이미지 로드
            x = img_to_array(img)     # numpy 배열로 변환
            x = np.expand_dims(x, axis=0)  # 배치 차원 추가
            
            # 원본 이미지 저장
            original_output_path = os.path.join(output_dir, f'original_{filename}')
            img.save(original_output_path)  # 원본 이미지 저장
        
            # 증강 이미지 생성 및 저장
            i = 0
            for batch in datagen.flow(x, batch_size=1, save_to_dir=output_dir, save_prefix=f'aug_{filename.split(".")[0]}', save_format='jpg'):
                i += 1
                if i >= 2:  # 하나의 이미지당 2개의 증강 이미지
                    break

In [4]:
base_dir = "/tf/Fixed_Data/Data_Initial"

In [5]:
# Apple_Fresh
input_dir = os.path.join(base_dir, 'apple/fresh')
output_dir = os.path.join(base_dir, 'Augmented_Data/Apple_Fresh')

data_augmentation(input_dir, output_dir)

In [6]:
# Apple_Rotten
input_dir = os.path.join(base_dir, 'apple/stale')
output_dir = os.path.join(base_dir, 'Augmented_Data/Apple_Rotten')

data_augmentation(input_dir, output_dir)

# Data Split
## Original & Augmented Data
/tf/Fixed_Data/Data_Initial/Augmented_Data  
│  
├── Apple_Fresh  
│  
└── Apple_Rotten  

## Splited Data
/tf/Fixed_Data/Data_Initial/Step1  
│  
├── train  
│&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├── Apple_Fresh  
│&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└── Apple_Rotten  
│  
├── validation  
│&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├── Apple_Fresh  
│&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└── Apple_Rotten  
│  
└── test  
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;├── Apple_Fresh  
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;└── Apple_Rotten  

    
## Split Ratio
train : validation : test = 7 : 2 : 1

## Dataset
|              | &nbsp;&nbsp;&nbsp; train &nbsp;&nbsp;&nbsp; | validation | &nbsp;&nbsp;&nbsp;&nbsp; test &nbsp;&nbsp;&nbsp;&nbsp; |
|:------------:|:------------:|:------------:|:------------:|
| Apple_Fresh  | 767 | 219 | 110 |
| Apple_Rotten | 965 | 275 | 139 |
| Total | 1732 | 494 | 249 |

## Code
교재 Chapter 8-2 서브셋 저장 코드 참고


In [7]:
import shutil
import random

### Source : 교재 코드 8-6

In [8]:
# 파일 리스트를 서브셋으로 저장
def make_subset(subset_name, file_list):
    
    for file_path, category in file_list:
        
        dst_dir = os.path.join(new_base_dir, subset_name, category)
        os.makedirs(dst_dir, exist_ok=True)
        shutil.copy(file_path, os.path.join(dst_dir, os.path.basename(file_path)))

In [9]:
# 주어진 비율로 데이터 분할 후 서브셋 생성
def split_data(base_dir, train_ratio, val_ratio):
    
    # 클래스 디렉토리 수집
    class_dirs = [d for d in os.listdir(base_dir) 
                  if os.path.isdir(os.path.join(base_dir, d)) and not d.startswith('.')]
    
    train_data = 0
    val_data = 0
    test_data = 0
    
    # 데이터 분할
    for class_dir in class_dirs:
        
        class_path = os.path.join(base_dir, class_dir)
        
        # 파일 목록 수집
        files = [(os.path.join(class_path, f), class_dir) 
                 for f in os.listdir(class_path) if not f.startswith('.')]
        
        # 데이터 shuffle
        random.shuffle(files)

        # 데이터 분할
        total = len(files)
        train_index = int(total * train_ratio)
        val_index = train_index + int(total * val_ratio)

        train_files = files[:train_index]
        val_files = files[train_index:val_index]
        test_files = files[val_index:]

        # 서브셋 생성
        make_subset("train", train_files)
        make_subset("validation", val_files)
        make_subset("test", test_files)
        
        train_data += len(train_files)
        val_data += len(val_files)
        test_data += len(test_files)

    return train_data, val_data, test_data

In [10]:
original_dir = "/tf/Fixed_Data/Data_Initial/Augmented_Data"
new_base_dir = "/tf/Fixed_Data/Data_Initial/Step1"

In [11]:
# train:validation:test = 7:2:1
train_ratio = 0.7
val_ratio = 0.2

In [12]:
# 데이터 분할 및 서브셋 생성
train_count, val_count, test_count = split_data(original_dir, train_ratio, val_ratio)

print(f"Train files: {train_count}, Validation files: {val_count}, Test files: {test_count}")

Train files: 1732, Validation files: 494, Test files: 249
